In [51]:
import math
import copy
fread = open("airplane.txt", "r") #netdata.txt
G_v_e = fread.readline()
G_v_e = G_v_e.split('\t') #change to \t if tab spaced
V = int(G_v_e[0])
E = int(G_v_e[1].strip())
V_list = list(range(0,V,1))
lines = fread.readlines()
A_dict = {}
G_mat = [ [ 0 for i in range(4) ] for j in range(E) ]
g = 0
for line in lines:
    t_h = line.split('\t') #change to \t if tab spaced
    print(t_h)
    G_mat[g][0] = int(t_h[0])
    G_mat[g][1] = int(t_h[1])
    G_mat[g][2] = int(t_h[2])
    G_mat[g][3] = int(t_h[3].strip())
    g += 1
for i in range(E):
    tail = G_mat[i][0]
    head = G_mat[i][1]
    if tail in A_dict.keys():
        A_dict[tail].append(head)        
    else:
        A_dict[tail] = [head]

capacity = [[0 for i in range(V)] for j in range(V)]
cost = [[math.inf for i in range(V)] for j in range(V)]
for i in range(E):
    tail = G_mat[i][0]
    head = G_mat[i][1]
    cost[tail][head] = G_mat[i][2]
    capacity[tail][head] = G_mat[i][3]

print(G_mat, "\n")
print(A_dict)
print(cost)
print(capacity)
fread.close()

['0', '1', '0', '150\n']
['0', '2', '0', '120\n']
['0', '3', '0', '140\n']
['0', '4', '0', '75\n']
['0', '5', '0', '25\n']
['0', '6', '0', '150\n']
['0', '7', '0', '20\n']
['0', '8', '0', '50\n']
['0', '9', '0', '45\n']
['0', '10', '0', '20\n']
['1', '11', '-4100', '10000\n']
['1', '12', '0', '10000\n']
['2', '11', '-8310', '10000\n']
['2', '13', '0', '10000\n']
['3', '11', '-5200', '10000\n']
['3', '14', '0', '10000\n']
['4', '11', '-5900', '10000\n']
['4', '15', '0', '10000\n']
['5', '12', '-5900', '10000\n']
['5', '13', '0', '10000\n']
['6', '12', '-3500', '10000\n']
['6', '14', '0', '10000\n']
['7', '12', '-7100', '10000\n']
['7', '15', '0', '10000\n']
['8', '13', '-4500', '10000\n']
['8', '14', '0', '10000\n']
['9', '13', '-5400', '10000\n']
['9', '15', '0', '10000\n']
['10', '14', '-2000', '10000\n']
['10', '15', '0', '10000\n']
['12', '16', '0', '150\n']
['13', '16', '0', '145\n']
['14', '16', '0', '340\n']
['15', '16', '0', '160\n']
['11', '12', '0', '180\n']
['12', '13', '0', 

In [52]:
def create_residual_network(A_dict, capacity, cost, V):
    R = []
    enode = V
    visited = [[0 for i in range(V)] for j in range(V)]
    for i in A_dict.keys():
        for j in A_dict[i]:
            #print(i, j)
            if (j in A_dict.keys()):
                if (visited[i][j] == 0):
                    if (i in A_dict[j]):
                        #add two nodes
                        R.append([i, enode, cost[i][j], capacity[i][j]])
                        #R.append([enode, i, capacity[i][j], cost[i][j]])
                        R.append([enode, j, 0, capacity[i][j]])
                        enode = enode + 1
                        R.append([j, enode, cost[j][i], capacity[j][i]])
                        R.append([enode, i, 0, capacity[j][i]]) 
                        enode = enode + 1
                    else:
                        R.append([i, j,  cost[i][j], capacity[i][j]])
                else:
                    continue
            else:
                R.append([i, j,  cost[i][j], capacity[i][j]])
            #print(R)
            visited[i][j] = 1
            visited[j][i] = 1
    #print(R)
    itr = len(R)
    #print(itr)
    for i in range(itr):
        t = R[i][0]
        h = R[i][1]
        ncost = -1*R[i][2]
        R.append([h, t, ncost, 0])
    return R, enode

In [53]:
def retrieve_flow_1(R_orig, R_capacity):
    flow = []
    for i in range(len(R_orig)):
        t = R_orig[i][0]
        h = R_orig[i][1]
        old = R_orig[i][3]
        new = R_capacity[t][h]
        if (old > new):
            flow.append(old - new)
        else:
            flow.append(0)            
    return flow


# Develop modified dictionary
def mod_dict(R, capacity):
    R_dict = {}
    for e in range(len(R)):
        t = R[e][0]
        h = R[e][1]
        if (capacity[t][h] != 0):
            if (t in R_dict.keys()):
                if (h not in R_dict[t]):
                    R_dict[t].append(h)
            else:
                R_dict[t] = [h]
        else:
            if (t in R_dict.keys()):
                if (h in R_dict[t]):
                    R_dict[t].remove(h)
    return R_dict


In [54]:
def pc_Bellman_Ford(R, V, s, t, R_capacity):
    parent = {}
    for i in range(V):
        parent[i] = i
    #print(parent)
    dist = [math.inf]*V
    dist[s] = 0
    changed_flag = True
    itr = 1
    while (changed_flag):
        changed_flag = False
        d_copy = dist.copy()  #remove this for modified
        for i in range(len(R)):
            tail = R[i][0]
            head = R[i][1]
            if (d_copy[head] > dist[tail] + R[i][2] and R_capacity[tail][head] > 0): #d_copy[head] to dist[head]
                d_copy[head] = dist[tail] + R[i][2] #d_copy[head] to dist[head]
                parent[head] = tail
                changed_flag = True
        dist = d_copy #remove this for modified
        #print(itr)
        #print(dist)
        #print(parent)
        itr = itr + 1
        if (itr > V): #to take care of negative cycles
            break    
    path = [t]
    curr = t
    while (curr != s and curr != parent[curr]):
        curr = parent[curr]
        path.insert(0, curr)
    if (path[0] != s):
        path = []
    return dist[t], path

In [55]:
#capacity scaling
def capacity_scaling(R, V, R_capacity, R_cost, s, t, f):
    R_orig = copy.deepcopy(R)
    flow_cost = 0
    remaining = f
    dist, path = pc_Bellman_Ford(R, V, s, t, R_capacity)
    flow = []
    while (len(path)!= 0 and remaining > 0):
        cmin = remaining
        for i in range(len(path)-1):
            tail = path[i]
            head = path[i+1]
            if (R_capacity[tail][head] < cmin):
                cmin = R_capacity[tail][head]
        remaining = remaining - cmin
        flow_cost = flow_cost + cmin*dist
        
        #update capacity
        for i in range(len(path)-1):
            tail = path[i]
            head = path[i+1]
            R_capacity[tail][head] =  R_capacity[tail][head] - cmin
            R_capacity[head][tail] =  R_capacity[head][tail] + cmin
        
        # if remaining left
        if (remaining > 0):
            dist, path = pc_Bellman_Ford(R, V, s, t, R_capacity)
    
    if (remaining == 0):
        flow = retrieve_flow_1(R_orig, R_capacity)
        return flow, flow_cost
    else:
        print("Can not send flow f")
        return flow, math.inf()
        
            
        
            
            
        
    

In [56]:
s = 0
t = 16
capacity = [[0 for i in range(V)] for j in range(V)]
cost = [[math.inf for i in range(V)] for j in range(V)]
for i in range(E):
    tail = G_mat[i][0]
    head = G_mat[i][1]
    cost[tail][head] = G_mat[i][2]
    capacity[tail][head] = G_mat[i][3]
    
R, Vnew = create_residual_network(A_dict, capacity, cost, V)

#print(Vnew)
R_capacity = [[0 for i in range(Vnew)] for j in range(Vnew)]
R_cost = [[math.inf for i in range(Vnew)] for j in range(Vnew)]
for i in range(len(R)):
    tail = R[i][0]
    head = R[i][1]
    R_cost[tail][head] = R[i][2]
    R_capacity[tail][head] = R[i][3]

for i in range(len(R)):
    print(R[i])
R_dict = mod_dict(R, R_capacity)
print(R_dict)

[0, 1, 0, 150]
[0, 2, 0, 120]
[0, 3, 0, 140]
[0, 4, 0, 75]
[0, 5, 0, 25]
[0, 6, 0, 150]
[0, 7, 0, 20]
[0, 8, 0, 50]
[0, 9, 0, 45]
[0, 10, 0, 20]
[1, 11, -4100, 10000]
[1, 12, 0, 10000]
[2, 11, -8310, 10000]
[2, 13, 0, 10000]
[3, 11, -5200, 10000]
[3, 14, 0, 10000]
[4, 11, -5900, 10000]
[4, 15, 0, 10000]
[5, 12, -5900, 10000]
[5, 13, 0, 10000]
[6, 12, -3500, 10000]
[6, 14, 0, 10000]
[7, 12, -7100, 10000]
[7, 15, 0, 10000]
[8, 13, -4500, 10000]
[8, 14, 0, 10000]
[9, 13, -5400, 10000]
[9, 15, 0, 10000]
[10, 14, -2000, 10000]
[10, 15, 0, 10000]
[12, 16, 0, 150]
[12, 13, 0, 180]
[13, 16, 0, 145]
[13, 14, 0, 180]
[14, 16, 0, 340]
[14, 15, 0, 180]
[15, 16, 0, 160]
[11, 12, 0, 180]
[1, 0, 0, 0]
[2, 0, 0, 0]
[3, 0, 0, 0]
[4, 0, 0, 0]
[5, 0, 0, 0]
[6, 0, 0, 0]
[7, 0, 0, 0]
[8, 0, 0, 0]
[9, 0, 0, 0]
[10, 0, 0, 0]
[11, 1, 4100, 0]
[12, 1, 0, 0]
[11, 2, 8310, 0]
[13, 2, 0, 0]
[11, 3, 5200, 0]
[14, 3, 0, 0]
[11, 4, 5900, 0]
[15, 4, 0, 0]
[12, 5, 5900, 0]
[13, 5, 0, 0]
[12, 6, 3500, 0]
[14, 6, 0, 0]


In [57]:
f = 795 #input flow
flow, flow_cost = capacity_scaling(R, Vnew, R_capacity, R_cost, s, t, f)
# Flow_cost
print('Total flow cost: ', flow_cost)
# Flow thru arcs
print('Flow through arcs-')
flow_value = 0
for i in range(len(R)):
    print('(', R[i][0], ',' , R[i][1], ')', ':', flow[i])

Total flow cost:  -2093200
Flow through arcs-
( 0 , 1 ) : 150
( 0 , 2 ) : 120
( 0 , 3 ) : 140
( 0 , 4 ) : 75
( 0 , 5 ) : 25
( 0 , 6 ) : 150
( 0 , 7 ) : 20
( 0 , 8 ) : 50
( 0 , 9 ) : 45
( 0 , 10 ) : 20
( 1 , 11 ) : 60
( 1 , 12 ) : 90
( 2 , 11 ) : 120
( 2 , 13 ) : 0
( 3 , 11 ) : 0
( 3 , 14 ) : 140
( 4 , 11 ) : 0
( 4 , 15 ) : 75
( 5 , 12 ) : 25
( 5 , 13 ) : 0
( 6 , 12 ) : 15
( 6 , 14 ) : 135
( 7 , 12 ) : 20
( 7 , 15 ) : 0
( 8 , 13 ) : 50
( 8 , 14 ) : 0
( 9 , 13 ) : 45
( 9 , 15 ) : 0
( 10 , 14 ) : 20
( 10 , 15 ) : 0
( 12 , 16 ) : 150
( 12 , 13 ) : 180
( 13 , 16 ) : 145
( 13 , 14 ) : 130
( 14 , 16 ) : 340
( 14 , 15 ) : 85
( 15 , 16 ) : 160
( 11 , 12 ) : 180
( 1 , 0 ) : 0
( 2 , 0 ) : 0
( 3 , 0 ) : 0
( 4 , 0 ) : 0
( 5 , 0 ) : 0
( 6 , 0 ) : 0
( 7 , 0 ) : 0
( 8 , 0 ) : 0
( 9 , 0 ) : 0
( 10 , 0 ) : 0
( 11 , 1 ) : 0
( 12 , 1 ) : 0
( 11 , 2 ) : 0
( 13 , 2 ) : 0
( 11 , 3 ) : 0
( 14 , 3 ) : 0
( 11 , 4 ) : 0
( 15 , 4 ) : 0
( 12 , 5 ) : 0
( 13 , 5 ) : 0
( 12 , 6 ) : 0
( 14 , 6 ) : 0
( 12 , 7 ) : 0
( 1